In [ ]:
def print_trainable_parameters(model):

    trainable_params = 0
    all_params = 0

    for _ , param in model.named_parameters():
        all_params += param.numel()

        if param.requires_grad:
            trainable_params += param.numel()

        print(f"trainable_params : ", {trainable_params})

In [ ]:
!pip install datasets huggingface_hub peft accelerate transformers bitsandbytes

In [ ]:
import gc
import torch

#del model  # or any other large objects
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import PeftModel, PeftConfig,  prepare_model_for_kbit_training,  LoraConfig, get_peft_model
from datasets import Dataset


from huggingface_hub import login
login(token="SECRET KEY")

device_map = {"": 0}
model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_use_double_quant = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.bfloat16
        )

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


config = LoraConfig(
        r = 8,
        lora_alpha = 32,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout = 0.05,
        bias = "none",
        task_type = "CAUSAL_LM"
        )

model = get_peft_model(model, config)
print_trainable_parameters(model)

with open("ocr.txt", "r") as file:
    lines = file.read().split("\n")

lines = [line for line in lines if line.strip() != ""]
raw_dataset = Dataset.from_dict({"text": lines})

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

trainer = Trainer (
        model = model ,
        train_dataset = tokenized_dataset,
        args = TrainingArguments (
            per_device_train_batch_size = 7,
            gradient_accumulation_steps = 4,
            warmup_steps = 2,
            num_train_epochs=20,  #
            #max_steps = 10,
            learning_rate = 2e-4,
            fp16 = True,
            logging_steps = 100,
            output_dir = "output_dir",
            optim = "paged_adamw_8bit",
            ),
        data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

model_to_save = trainer.model.module if hasattr(trainer.model, "module") else trainer.model
model_to_save.save_pretrained("Mobitel")

In [ ]:
model = model.to("cuda:0")
model.eval()

while True:
    question = input("Enter a question here: ")
    inputs = tokenizer(question, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            top_p=0.95,
            temperature=0.7
        )

    print(tokenizer.decode(output[0], skip_special_tokens=True))